<a href="https://colab.research.google.com/github/DarShabi/OCR-Company-Project/blob/main/Week%234/textbox_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!sudo apt-get install -q tesseract-ocr
#!pip install -q pytesseract Pillow
!pip install -q transformers

hi

In [2]:
!pip install -q sentencepiece


In [3]:
from PIL import Image, ImageDraw
import pytesseract
from pytesseract import Output

In [4]:
def is_inside(box1, box2):
    """Check if box1 is inside box2."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    return x1 > x2 and y1 > y2 and x1 + w1 < x2 + w2 and y1 + h1 < y2 + h2

image_path = '/content/38c9b7f0571b5421e78a3226c0d3eee0-2.jpg'
image = Image.open(image_path)
draw = ImageDraw.Draw(image)

# Use Tesseract with a specific Page Segmentation Mode
data = pytesseract.image_to_data(image, config='--psm 11', output_type=Output.DICT)

confidence_threshold = 70  # Set your desired threshold here
boxes = [(data['left'][i], data['top'][i], data['width'][i], data['height'][i]) for i in range(len(data['text'])) if int(data['conf'][i]) > confidence_threshold]
filtered_boxes = []

# Filter out nested boxes
for box in boxes:
    if not any(is_inside(box, other_box) for other_box in boxes):
        filtered_boxes.append(box)

# Draw filtered bounding boxes on the image
for box in filtered_boxes:
    x, y, w, h = box
    draw.rectangle([x, y, x + w, y + h], outline='red')

# Save the image with filtered bounding boxes
output_path = 'filtered_output_image5.jpg'
image.save(output_path)
print(f"Image saved to {output_path}")

Image saved to filtered_output_image5.jpg


In [5]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch

# Initialize the TrOCR model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-stage1")
trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-stage1")
trocr_model.eval()

def extract_text_with_trocr(image, boxes):
    texts = []
    for box in boxes:
        x, y, w, h = box
        cropped_image = image.crop((x, y, x + w, y + h))

        # Process the cropped image and predict text using TrOCR
        inputs = processor(cropped_image, return_tensors="pt")

        # Provide initial input for the decoder using the padding token as the start token
        decoder_input_ids = torch.full((1, 1), processor.tokenizer.pad_token_id, dtype=torch.long)
        outputs = trocr_model(decoder_input_ids=decoder_input_ids, pixel_values=inputs["pixel_values"])

        predicted_text = processor.decode(outputs.logits.argmax(-1)[0])
        texts.append(predicted_text)

    return texts



Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-stage1 and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Extract text from the bounding boxes using TrOCR
texts = extract_text_with_trocr(image, filtered_boxes)
print(texts)

['.', '.', '.', 'us', 'writing', 'Service', 'l', 'li', '</s>', 'ad', 'p', '</s>', 'o', "'", '.', '.']
